# Transfer learning met PyTorch

In deze notebook gaan we het concept van transfer learning onderzoeken. Transfer learning is een techniek waarbij een pre-trained neuraal netwerk wordt gebruikt als startpunt voor een nieuwe taak. Er zijn twee belangrijke benaderingen binnen transfer learning:

1. **Feature Extractie**: Hierbij houden we de gewichten van het pre-trained model vast en gebruiken we het model als feature extractor. Alleen het laatste (geclassificeerde) deel van het netwerk wordt aangepast aan de nieuwe taak.
  
2. **Fine-Tuning**: Hierbij worden de gewichten van (een deel van) het pre-trained model verder getraind op de nieuwe taak. Dit maakt het mogelijk om het model verder aan te passen aan de specifieke data.

## Stappen in de notebook

1. **Data voorbereiding**:
   - We gebruiken een kleine dataset voor classificatie (bijvoorbeeld een subset van de CIFAR-10 dataset).

2. **Model laden en voorbereiden**:
   - We laden een pre-trained model (bijv. ResNet-18) uit PyTorch's `torchvision` bibliotheek.
   - We maken aanpassingen voor beide benaderingen van transfer learning.

3. **Training**:
   - We demonstreren zowel de **feature extractie** als de **fine-tuning** methode.
   - We trainen de laatste laag voor de feature extractie en een deel van het netwerk voor fine-tuning.

4. **Evaluatie**:
   - We evalueren de prestaties van beide benaderingen op een testset.

---

## 1. Data Voorbereiding

We beginnen met het importeren van de benodigde bibliotheken en het laden van de dataset.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import torchvision.models as models # hier zitten de pretrained netwerken
import random

# Data transformaties
transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 pixels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# transform = models.ResNet18_Weights.IMAGENET1K_V1.transforms  # gebruik de pretrained transform

# Laden van de CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
total_samples = len(trainset)
sample_size = int(0.01 * total_samples)   # reduceer dataset naar 1% van het originele
random_indices = random.sample(range(total_samples), sample_size)
trainset_reduced = Subset(trainset, random_indices)
trainloader = DataLoader(trainset_reduced, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
total_samples = len(testset)
sample_size = int(0.01 * total_samples) # reduceer dataset naar 1% van het originele
random_indices = random.sample(range(total_samples), sample_size)
testset_reduced = Subset(testset, random_indices)

# Create a DataLoader for the reduced dataset
testloader = DataLoader(testset_reduced, batch_size=32, shuffle=True, num_workers=2)

# Klassen in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:08<00:00, 20065213.03it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## 2. Model Laden en Voorbereiden

### A) Feature Extractie

In deze aanpak houden we de gewichten van het pre-trained model vast en passen we alleen de laatste laag aan.

In [5]:
model_fe = models.resnet18(models.ResNet18_Weights.DEFAULT)

# in de print hieronder zagen we dat dit model in de laatste laag een fully-connected gedeelte heeft met 1000 output neuronen
# we willen dat dit er maar 10 zijn voor CIFAR10

# train de huidige lagen niet verder
for param in model_fe.parameters():
    param.requires_grad=False  # analoog aan de no_grad functie

# pas de laatste laag aan
num_in = model_fe.fc.in_features # aantal in_features van de laatste laag
model_fe.fc = nn.Linear(num_in, 10) # 10 want cifar10

print(model_fe)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### B) Fine-Tuning

In deze aanpak staan we toe dat sommige (of alle) gewichten verder worden getraind.

In [10]:
model_ft = models.resnet18(models.ResNet18_Weights.DEFAULT)

# in de print hieronder zagen we dat dit model in de laatste laag een fully-connected gedeelte heeft met 1000 output neuronen
# we willen dat dit er maar 10 zijn voor CIFAR10

# train de huidige lagen niet verder
for name, param in model_ft.named_parameters():
    if "layer4" in name or 'fc' in name:
        param.requires_grad=True # train vanaf de laatste convolutionele blok
    else:
        param.requires_grad=False # alles ervoor wordt niet getrained

# pas de laatste laag aan
num_in = model_ft.fc.in_features # aantal in_features van de laatste laag
model_ft.fc = nn.Linear(num_in, 10) # 10 want cifar10

print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 3. Training

Hier zullen we beide modellen trainen, eerst het model voor feature extractie, gevolgd door het model voor fine-tuning.

### A) Feature Extractie Training

In [1]:

from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights

weights = FCN_ResNet50_Weights.DEFAULT
model = fcn_resnet50(weights=weights)

print(model)

Downloading: "https://download.pytorch.org/models/fcn_resnet50_coco-1167a1af.pth" to C:\Users\jens.baetens3/.cache\torch\hub\checkpoints\fcn_resnet50_coco-1167a1af.pth
100%|██████████| 135M/135M [00:04<00:00, 33.1MB/s] 


FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [9]:
import time

criterion = nn.CrossEntropyLoss() # voor multi-class classificatie
optimizer_fe = optim.Adam(model_fe.fc.parameters(), lr=0.001) # let op: hier enkel de gewichten die je wil gaan optimaliseren

num_epochs=5
model_fe.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    start_time = time.time()

    for inputs, labels in trainloader:
        optimizer_fe.zero_grad()
        outputs = model_fe(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_fe.step()

        running_loss += loss.item()

    end_time = time.time()
    print(f"Epoch {epoch}/{num_epochs}: loss {running_loss/len(trainloader)}, time {end_time - start_time}")

Epoch 0/5: loss 2.2340712919831276, time 11.068679571151733
Epoch 1/5: loss 1.7835159376263618, time 7.572076320648193
Epoch 2/5: loss 1.4819212332367897, time 7.3384690284729
Epoch 3/5: loss 1.2934693917632103, time 7.406125545501709
Epoch 4/5: loss 1.1437012702226639, time 7.324072599411011


### B) Fine-Tuning Training

In [11]:
import time

criterion = nn.CrossEntropyLoss() # voor multi-class classificatie
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001) # in dit geval gemakkelijker om alles te zeggen

num_epochs=5
model_ft.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    start_time = time.time()

    for inputs, labels in trainloader:
        optimizer_ft.zero_grad()
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_ft.step()

        running_loss += loss.item()

    end_time = time.time()
    print(f"Epoch {epoch}/{num_epochs}: loss {running_loss/len(trainloader)}, time {end_time - start_time}")

Epoch 0/5: loss 1.3705161437392235, time 9.344644546508789
Epoch 1/5: loss 0.25629299134016037, time 9.318185806274414
Epoch 2/5: loss 0.06128747668117285, time 9.43606162071228
Epoch 3/5: loss 0.038651138136629015, time 9.504300594329834
Epoch 4/5: loss 0.03377546390402131, time 9.718537330627441


## 4. Evaluatie
Na het trainen evalueren we de prestaties van beide modellen op de testset.

### A) Evaluatie van het Feature Extractie Model

In [12]:
correct = 0
total = len(testloader)

model_fe.eval() # evaluatie modus (bereken geen gewichten optimalisaties en dergelijke)
with torch.no_grad():
    for inputs,labels in testloader:
        outputs = model_fe(inputs)
        _, predicted_classes = torch.max(outputs, 1) # in pytorch is max ook een argmax
        correct += (predicted_classes == labels).sum().item()

print(f"Accuracy is {correct/total}")

Accuracy is 15.5


### B) Evaluatie van het Fine-Tuning Model

In [13]:
correct = 0
total = len(testloader)

model_ft.eval() # evaluatie modus (bereken geen gewichten optimalisaties en dergelijke)
with torch.no_grad():
    for inputs,labels in testloader:
        outputs = model_ft(inputs)
        _, predicted_classes = torch.max(outputs, 1) # in pytorch is max ook een argmax
        correct += (predicted_classes == labels).sum().item()

print(f"Accuracy is {correct/total}")

Accuracy is 16.75


## Oefening: transfer learning met extra lagen voor FashionMNIST

### Oefeningomschrijving:
In deze oefening ga je een neuraal netwerk trainen met de FashionMNIST dataset, gebruikmakend van transfer learning. Je zult een pre-trained ResNet-18 model gebruiken, en daar drie extra volledig verbonden lagen aan toevoegen. Vervolgens train je het model en evalueer je de prestaties.

### Stappen:

**Data Voorbereiding:**

Laad de FashionMNIST dataset en breng de nodige transformaties aan.
Splits de data in een trainingsset en een testset.

**Model Voorbereiding:**

Laad een pre-trained ResNet-18 model.
Pas het model aan door drie extra volledig verbonden lagen toe te voegen:
* De eerste extra laag moet 512 neuronen hebben en een ReLU activatiefunctie.
* De tweede extra laag moet 256 neuronen hebben en een ReLU activatiefunctie.
* De derde extra laag moet 128 neuronen hebben en een ReLU activatiefunctie.
* De laatste output laag moet het aantal klassen in FashionMNIST (10 klassen) bevatten.

**Training:**

Definieer een loss-functie en een optimizer.
Train het model voor een aantal epochs.
Meet en print de tijd die elke epoch kost.

**Evaluatie:**

Evalueer de prestaties van het getrainde model op de testset en rapporteer de accuraatheid.

**Vragen om te beantwoorden:**

* Wat is het effect van de extra lagen op de prestaties van het model?
* Hoeveel tijd kost elke epoch en hoe varieert dit met het aantal lagen?